<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/Step_2_Seeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Elephant Oracle Notebook

You've completed the setup and are ready to submit real estate data to the Elephant protocol. This notebook will transform your seed data into validated, blockchain-ready submissions.

## What This Notebook Does

This interactive notebook automates:
- Converting seed data to lexicon format
- Validating against Elephant schemas
- Canonicalizing data for consistent hashing
- Uploading to IPFS via Pinata
- Preparing transaction data for blockchain submission

## What You'll Do

1. **Configure Pinata** (1 minute)
   - Enter your JWT token

2. **Transform Data** (2 minutes)
   - Run conversion to lexicon format
   - Auto-validate against [lexicon.elephant.xyz](https://lexicon.elephant.xyz/) schemas
   - Generate canonical JSON

3. **Upload to IPFS** (2 minutes)
   - Submit validated data to IPFS
   - Receive content identifiers (CIDs)

4. **Prepare Submission** (1 minute)
   - Generate upload-results.csv
   - Download for oracle portal

5. **Submit to Blockchain** (2 minutes)
   - Visit oracle.elephant.xyz
   - Upload results file
   - Confirm MetaMask transactions

## Your Impact

Each submission contributes to consensus. When three oracles submit matching data hashes, the data becomes blockchain truth and participants receive vMahout governance tokens.

Let's begin.

In [1]:
# @title Step 1: Please enter your property parcel ID, address and request used to get this property (you can add as much as needed just insert property information then run the cell)
import os
parcel_id = "" # @param {"type":"string"}
address = "" # @param {"type":"string"}
request_method = "" # @param {"type":"string"}
url = "" # @param {"type":"string"}
County = "" # @param {"type":"string"}
headers = "" # @param {"type":"string"}

os.environ["parcel_id"] = parcel_id
os.environ["address"] = address
os.environ["request_method"] = request_method
os.environ["url"] = url
os.environ["County"] = County
os.environ["headers"] = headers


import json
import os
import re
import sys
from urllib.parse import urlparse, parse_qs

# Get data from Step 2 environment variables
parcel_id = os.environ.get("parcel_id", "")
address = os.environ.get("address", "")
request_method = os.environ.get("request_method", "")
url = os.environ.get("url", "")
county = os.environ.get("County", "")
headers = os.environ.get("headers", "")


def is_empty_value(value):
    """Check if value is empty or None"""
    if value is None:
        return True
    if isinstance(value, str) and value.strip() == "":
        return True
    return False

def ensure_directory(file_path):
    """Ensure the directory for the file exists"""
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

def extract_query_params_and_base_url(url):
    """Extract query parameters and base URL separately"""
    if is_empty_value(url):
        return None, None

    try:
        parsed_url = urlparse(url)

        # Base URL without query parameters
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

        # Query parameters as multiValueQueryString format (object of arrays)
        query_params = parse_qs(parsed_url.query)
        multi_value_query = dict(query_params) if query_params else None

        return base_url, multi_value_query
    except Exception as e:
        print(f"Warning: Could not parse URL: {e}")
        return url, None

def create_parcel_folder(parcel_id, address, method, url, county, headers):
    # Create folder name based on parcel_id
    clean_parcel_id = re.sub(r"[^\w\-_]", "_", str(parcel_id))
    folder_name = f"output/{clean_parcel_id}"
    ensure_directory(folder_name + "/")

    # Extract base URL and query parameters separately
    base_url, multi_value_query = extract_query_params_and_base_url(url)

    # Create unnormalized_address.json
    unnormalized_address_data = {
        "full_address": address if not is_empty_value(address) else None,
        "source_http_request": {
            "method": method if not is_empty_value(method) else None,
            "url": base_url if not is_empty_value(base_url) else None,
            "multiValueQueryString": multi_value_query
        },
        "county_jurisdiction": county if not is_empty_value(county) else None,
        "request_identifier": parcel_id if not is_empty_value(parcel_id) else None,
    }
    if headers and not is_empty_value(headers):
        unnormalized_address_data["source_http_request"]["headers"] = headers

    # Create property_seed.json
    property_seed_data = {
        "parcel_id": parcel_id if not is_empty_value(parcel_id) else None,
        "source_http_request": {
            "method": method if not is_empty_value(method) else None,
            "url": base_url if not is_empty_value(base_url) else None,
            "multiValueQueryString": multi_value_query
        },
        "request_identifier": parcel_id if not is_empty_value(parcel_id) else None,
    }
    if headers and not is_empty_value(headers):
        property_seed_data["source_http_request"]["headers"] = headers

    # Create relationship_property_to_address.json
    relationship_data = {
        "from": {"/": "./property_seed.json"},
        "to": {"/": "./unnormalized_address.json"}
    }

    # Create root schema
    root_schema = {
        "label": "Seed",
        "relationships": {"property_seed": {"/": "./relationship_property_to_address.json"}},
    }

    # Write all JSON files
    files_to_create = [
        (f"{folder_name}/unnormalized_address.json", unnormalized_address_data),
        (f"{folder_name}/property_seed.json", property_seed_data),
        (f"{folder_name}/relationship_property_to_address.json", relationship_data),
        (f"{folder_name}/bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe.json", root_schema),
    ]

    for filename, data_obj in files_to_create:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data_obj, f, indent=2, ensure_ascii=False)

    return folder_name, unnormalized_address_data, property_seed_data

def process_input_data():
    try:
        # Validate required data
        if is_empty_value(parcel_id):
            print("❌ Error: parcel_id is required but not provided")
            return

        # Show extracted URL components
        base_url, multi_value_query = extract_query_params_and_base_url(url)

        # Create parcel folder and files
        folder_name, address_data, property_data = create_parcel_folder(
            parcel_id, address, request_method, url, county, headers
        )

        print(f"\n✅ property with parcel {parcel_id} added successfully")

    except Exception as e:
        print(f"❌ Error processing input data: {e}")
        import traceback
        traceback.print_exc()

# Process the input data
process_input_data()


✅ property with parcel 30434108090030050 added successfully


## Step 2: Upload .env file


| Variable Name           | Purpose                     |
|-------------------------|-----------------------------|
| `PINATA_JWT`     | Access to pinata key              |


- Click the **folder icon** 📂 in the left sidebar to open the file browser.
- Then click the **"Upload"** button and choose your `.env` file.

```env
# example of .env file
PINATA_JWT=xxxxx
```


In [2]:
# @title Step 3: Run to upload your files to the IPFS
! pip3 install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()
!npx -y @elephant-xyz/cli validate-and-upload output

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹🐘 Elephant Network CLI - Validate and Upload

7Initializing    |████████████████████████████████████████| 100% | 0/0 | Errors: 0 | Skipped: 0 | 0s | ETA: 0s7Fetching Schemas |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: 0sPre-fetching Schemas |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsFetching Schemas |████████████████████████████████████████| 100% | 1/1 | Errors: 0 | Skipped: 0 | 0s | ETA: 0s8
Processing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 1s | ETA: NFsProcessing Files |████████████████████████████████████████| 100% | 1/1 | Errors: 0 | Skipped: 0 | 1s | ETA: 0sInstalling/Updating Fact Sheet Tool |██████████

## Step 4: Submitting Your Data to the Blockchain

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

In [4]:
!npx -y @elephant-xyz/cli submit-to-contract upload-results.csv

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋🐘 Elephant Network CLI - Submit to Contract

7Indexing on-chain data |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: 0sSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 1s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 2s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 3s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 4s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 5s | ETA: NFsSubmitting Transactions |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 6s | ETA: NFsSubmitting Transa